In [63]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from datetime import datetime, timedelta,date
from mapper import mapper_ru,mapper_global

In [145]:
def pre_global(df, columns_uniq):
    df = pd.concat([df.iloc[:,1:2], df.iloc[:,4:]],axis=1)
#     print(df)
    for country in columns_uniq:
#         print(country)
        idx = df['Country/Region']==country
        idx_n =df['Country/Region']!=country
        df_ = pd.DataFrame((df[idx].iloc[:,1:].sum(axis=0))).T
        df_['Country/Region'] = country
        cols = df_.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        df_ = df_[cols]
        df = df[idx_n]
        df = pd.concat([df,df_],axis=0)
    df.index = np.arange(df.shape[0])
    return df

In [146]:
path_time_series = './data/COVID-19_plus_Russia/csse_covid_19_data/csse_covid_19_time_series/'
recovered_global_name = 'time_series_covid19_recovered_global.csv'
recovered_ru_name = 'time_series_covid19_confirmed_RU.csv'
deaths_global_name = 'time_series_covid19_deaths_global.csv'
deaths_ru_name = 'time_series_covid19_deaths_RU.csv'
path = './data/'
countries_data_name = 'countries.csv'
russia_regions_name = 'russia_regions.csv'
sample_submission = 'sample_submission.csv'

In [147]:
recovered_global = pd.read_csv(path_time_series+ recovered_global_name)
deaths_global = pd.read_csv(path_time_series + deaths_global_name)

In [148]:
# deaths_global.head()

In [149]:
# recovered_global.head()

In [150]:
# recovered_global = pd.concat([recovered_global.iloc[:,1:2], recovered_global.iloc[:,4:]],axis=1)
# deaths_global = pd.concat([deaths_global.iloc[:,1:2], deaths_global.iloc[:,4:]],axis=1)


In [151]:
uniq_rec = ['China', 'Australia', 'Denmark',"Demark", "France", 'Netherlands','United Kingdom']
recovered_global = pre_global(recovered_global,uniq_rec)
# for country in uniq_rec:
#     idx = recovered_global['Country/Region']==country
#     idx_n =recovered_global['Country/Region']!=country
#     df_ = pd.DataFrame((recovered_global[idx].iloc[:,1:].sum(axis=0))).T
#     df_['Country/Region'] = country
#     cols = df_.columns.tolist()
#     cols = cols[-1:] + cols[:-1]
#     df_ = df_[cols]
#     recovered_global = recovered_global[idx_n]
#     recovered_global = pd.concat([recovered_global,df_],axis=0)
# recovered_global.index = np.arange(recovered_global.shape[0])

In [152]:
uniq_global = ['China', 'Australia', 'Denmark',"Demark", "France", 'Netherlands','United Kingdom','Canada']
deaths_global = pre_global(deaths_global,uniq_global)
# for country in uniq_global:
#     idx = deaths_global['Country/Region']==country
#     idx_n =deaths_global['Country/Region']!=country
#     df_ = pd.DataFrame((deaths_global[idx].iloc[:,1:].sum(axis=0))).T
#     df_['Country/Region'] = country
#     cols = df_.columns.tolist()
#     cols = cols[-1:] + cols[:-1]
#     df_ = df_[cols]
#     deaths_global = deaths_global[idx_n]
#     deaths_global = pd.concat([deaths_global,df_],axis=0)
# deaths_global.index = np.arange(deaths_global.shape[0])

In [153]:
recovered_global.shape,deaths_global.shape

((186, 97), (186, 97))

In [10]:
def get_dataset_global(df):
    solution_df = pd.DataFrame({'name':df['Country/Region'].values})
    date = df.columns.values[1:]
    solution = pd.concat([solution_df,df[date]],axis=1)
    dataset = solution[date].values.T 
    return dataset

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    n = sequences.shape[0]
    X, y = list(), list()
    for i in range(n):
        # find the end of this pattern
        end_ix = i + n_steps
    # check if we are beyond the dataset
        if end_ix > n-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def build_model_simple(n_steps,n_features):
    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(100, activation='relu'))
    model.add(Dense(n_features))
    model.compile(optimizer='adam', loss='mse')
    return model

In [79]:
columns_global = recovered_global['Country/Region']
last_date = pd.to_datetime(recovered_global.columns.values[-1:]).date[0]


In [72]:
n_steps = 3
n_features = 186
model_rec = build_model_simple(n_steps,n_features)

dataset_rec = get_dataset_global(recovered_global) 
X_rec,y_rec = split_sequences(dataset_rec,n_steps)
hisoty_rec = model_rec.fit(X_rec, y_rec, epochs=10, verbose=1)

Epoch 1/10
93/93 [==============================] - 7s 73ms/step - loss: 46314054.0215
Epoch 2/10
93/93 [==============================] - 0s 2ms/step - loss: 45569262.9892
Epoch 3/10
93/93 [==============================] - 0s 2ms/step - loss: 45008087.8925
Epoch 4/10
93/93 [==============================] - 0s 2ms/step - loss: 44362355.2258
Epoch 5/10
93/93 [==============================] - 0s 2ms/step - loss: 43676842.9462
Epoch 6/10
93/93 [==============================] - 0s 2ms/step - loss: 42391915.4839
Epoch 7/10
93/93 [==============================] - 0s 2ms/step - loss: 40965719.8710
Epoch 8/10
93/93 [==============================] - 0s 2ms/step - loss: 39148694.4516
Epoch 9/10
93/93 [==============================] - 0s 2ms/step - loss: 37135267.4839
Epoch 10/10
93/93 [==============================] - 0s 2ms/step - loss: 34626434.3441


In [73]:

n_steps = 3
n_features = 186
model_deaths = build_model_simple(n_steps,n_features)

dataset_deaths = get_dataset_global(deaths_global) 
X_deaths,y_deaths = split_sequences(dataset_deaths,n_steps)
hisoty_deaths = model_deaths.fit(X_deaths, y_deaths, epochs=10, verbose=1)

Epoch 1/10
93/93 [==============================] - 7s 72ms/step - loss: 3766618.0000
Epoch 2/10
93/93 [==============================] - 0s 2ms/step - loss: 3648414.1519
Epoch 3/10
93/93 [==============================] - 0s 2ms/step - loss: 3480389.7581
Epoch 4/10
93/93 [==============================] - 0s 2ms/step - loss: 3373598.4194
Epoch 5/10
93/93 [==============================] - 0s 2ms/step - loss: 3230833.2849
Epoch 6/10
93/93 [==============================] - 0s 2ms/step - loss: 3113751.6425
Epoch 7/10
93/93 [==============================] - 0s 2ms/step - loss: 2968245.4489
Epoch 8/10
93/93 [==============================] - 0s 2ms/step - loss: 2785826.3414
Epoch 9/10
93/93 [==============================] - 0s 2ms/step - loss: 2591064.6505
Epoch 10/10
93/93 [==============================] - 0s 2ms/step - loss: 2361365.6532


In [94]:
class Prediction:
    def __init__(self, model,last_date,columns_df):
        self.columns_df = columns_df
        self.model = model
        self.last_date = last_date
        self.predion_date = self.__make_date_predition()
        self.size_prediction = len(self.predion_date)
        
    def predict(self,dataset):
        self.dateset = dataset
        matrix_prediction = self.__make_prediction_matrix()
        return self.__make_dataframe_prediction(matrix_prediction)
    
    def __make_dataframe_prediction(self,matrix_prediction):
        df = pd.DataFrame(matrix_prediction,columns=self.columns_df)
        df['date'] = self.predion_date
        return df
    def __make_prediction_matrix(self):
        dataset = self.dateset
        model = self.model
        count = self.size_prediction
        
        x = np.array([dataset[-3:,:]])
        x_i = dataset[-2:,:]
        result = []
        for _ in range(count):
            y_pred = model.predict(x)[0]
            y_pred_ = list(np.sign(y_pred)*(y_pred))
            y_pred_ = list((y_pred))
            result.append(y_pred_)
            y_pred = y_pred.reshape(1,186)
            x = np.vstack((x_i,y_pred))
            x_i = x[-2:,:]
            x = np.array([x])
        return result
        
    def __make_date_predition(self):
        
        last_date = self.last_date
        last_date += timedelta(days=1)
        res_date = []
#         print(last_date)
        while last_date< date(2021,1,1):
#             print(last_date)
            res_date.append(last_date)
            last_date += timedelta(days=1)
        return res_date
        
        
    
class Submission:
    def __init__(self,columns_df,date_prediction):
        self.columns_df = columns_df
        self.date_prediction = date_prediction
        
    def submit(self,rec_df,deaths_df,mapper):
        
        df_res = pd.DataFrame({}) 
        for column in self.columns_df:
            
            df_res_ = pd.DataFrame({}) 
            
            df_res_['date'] = self.date_prediction
            df_res_['region'] = [column]*len(self.date_prediction)
#             print(column)
            df_res_['prediction_confirmed'] = rec_df[column]
            
            df_res_['prediction_deaths'] = deaths_df[column]
            
            df_res = pd.concat([df_res,df_res_],axis=0)
        df_res.index=np.arange(df_res.shape[0])
        df_res.region = df_res.region.map(mapper)
        return df_res

In [95]:
predictioner_rec = Prediction(model_rec,last_date,columns_global)
df_pred_rec = predictioner_rec.predict(dataset_rec)

In [96]:
predictioner_deaths = Prediction(model_deaths,last_date,columns_global)
df_pred_deaths = predictioner_deaths.predict(dataset_deaths)

In [97]:
# columns_global

In [105]:
submissioner = Submission(columns_global,predictioner_rec.predion_date)
subm = submissioner.submit(df_pred_rec,df_pred_deaths,mapper_global)

In [106]:
# subm = subm.dropna()
subm = subm[subm.region!='RU']
subm.shape
# subm.dropna()

(46314, 4)

In [107]:
46314+ 21165

67479

In [108]:
subm

,date,region,prediction_confirmed,prediction_deaths
0,2020-04-27,AFG,-2706.972900,-601.489990
1,2020-04-28,AFG,-4989.375977,-1171.849976
2,2020-04-29,AFG,-6019.639160,121.193703
3,2020-04-30,AFG,-571.872314,457.996613
4,2020-05-01,AFG,-1063.609863,-204.914474
5,2020-05-02,AFG,-752.525757,84.117981
6,2020-05-03,AFG,582.876648,293.211487
7,2020-05-04,AFG,538.573669,-17.468050
8,2020-05-05,AFG,-888.029114,85.239937
9,2020-05-06,AFG,-149.557968,44.126705
